# Practica 2 - Natural Language processing

In [1]:
!pip install tensorflow

In [2]:
import numpy as np

import tensorflow as tf

import pandas as pd
from collections import Counter
import re

from tensorflow import keras


import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.utils import to_categorical

from tensorflow.keras import Model, Input, layers
from tensorflow.keras.layers import Embedding, Dot, Reshape, Dense

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import skipgrams

from keras.callbacks import TensorBoard

import re
from collections import Counter
from tensorflow.keras.preprocessing.text import Tokenizer

import numpy as np
from tensorflow.keras.preprocessing.sequence import skipgrams

In [3]:
import gdown
import os

# Crear el directorio de datos si no existe
data_dir = "data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# URL de Google Drive en formato correcto para gdown
url = "https://drive.google.com/uc?id=1GXSUzaXDvrlcimgTwRR9tmDr6xr8gu49"
zip_filename = "sentiment_analysis.zip"
zip_path = os.path.join(data_dir, zip_filename)

# Descargar el archivo
if not os.path.exists(zip_path):
    gdown.download(url, zip_path, quiet=False)
else:
    print("Data zipfile already exists")

Data zipfile already exists


In [5]:
import shutil
from zipfile import ZipFile
from concurrent.futures import ThreadPoolExecutor

data_dir = "data"
zip_filename = "sentiment_analysis.zip"
zip_path = os.path.join(data_dir, zip_filename)
files = ["train.csv", "test.csv"]
full_paths = [os.path.join(data_dir, file) for file in files]

if not all(os.path.isfile(path) for path in full_paths):
    with ZipFile(zip_path, 'r') as zf:
        with ThreadPoolExecutor() as exe:
            for file in zf.namelist():
                if not file.startswith("__MACOSX"):
                    exe.submit(zf.extract, file, path=data_dir)
else:
    print("test and train files already exist")

In [3]:
import os
import pandas as pd

data_dir = "data"
train_path = os.path.join(data_dir, "train.csv")
test_path = os.path.join(data_dir, "test.csv")
train_df = pd.read_csv(train_path, encoding='ISO-8859-1', index_col="textID")
test_df = pd.read_csv(test_path, encoding='ISO-8859-1', index_col="textID")
train_df.head()

,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
textID,,,,,,,,,
cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [4]:
train_df = train_df.dropna()
test_df = test_df.dropna()
text_df = pd.concat([train_df, test_df])
text_df.shape

(31014, 9)

In [5]:
from tensorflow.keras.layers import TextVectorization

vocab_size = 10_000

vectorize_layer = TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    standardize="lower_and_strip_punctuation",
    split="whitespace",
)

corpus = [text_df["text"].values[0]]

vectorize_layer.adapt(corpus)

vectorize_layer(corpus)

<tf.Tensor: shape=(1, 7), dtype=int64, numpy=array([[5, 7, 3, 4, 6, 2, 8]])>

In [6]:
vectorize_layer.get_vocabulary()[:10]

['',
 '[UNK]',
 np.str_('were'),
 np.str_('responded'),
 np.str_('if'),
 np.str_('id'),
 np.str_('i'),
 np.str_('have'),
 np.str_('going')]

In [10]:
import tensorflow as tf

train_ds = tf.data.Dataset.from_tensor_slices(train_df["text"])

print(train_ds.element_spec)
for sentence in train_ds.take(5):
  print(sentence)

TensorSpec(shape=(), dtype=tf.string, name=None)
tf.Tensor(b' I`d have responded, if I were going', shape=(), dtype=string)
tf.Tensor(b' Sooo SAD I will miss you here in San Diego!!!', shape=(), dtype=string)
tf.Tensor(b'my boss is bullying me...', shape=(), dtype=string)
tf.Tensor(b' what interview! leave me alone', shape=(), dtype=string)
tf.Tensor(b' Sons of ****, why couldn`t they put them on the releases we already bought', shape=(), dtype=string)


In [36]:
# Convert the Pandas column to a tf.data.Dataset
text_ds = tf.data.Dataset.from_tensor_slices([train_df["text"].values[0]])

# Apply the vectorization layer to get token sequences
vectorized_ds = text_ds.map(lambda x: vectorize_layer(x))

print(vectorized_ds.element_spec)
next(vectorized_ds.as_numpy_iterator())

TensorSpec(shape=(None,), dtype=tf.int64, name=None)


array([ 284,   17, 8527,   68,    2,  121,   47])

In [47]:
def generate_skipgram_pairs(sequence, window_size=4):
    seq_len = tf.shape(sequence)[0]
    positions = tf.range(seq_len)

    def extract_context(i):
        target = sequence[i]
        return tf.cond(
            tf.equal(target, 0),
            lambda: tf.data.Dataset.from_tensors((tf.constant(-1, dtype=sequence.dtype),
                                                  tf.constant(-1, dtype=sequence.dtype))).take(0),
            lambda: _valid_context(i, target)
        )

    def _valid_context(i, target):
        start = tf.maximum(0, i - window_size)
        end = tf.minimum(seq_len, i + window_size + 1)

        # Exclude center word
        left = sequence[start:i]
        right = sequence[i + 1:end]
        context = tf.concat([left, right], axis=0)

        # Remove zeros
        non_zero = tf.not_equal(context, 0)
        context = tf.boolean_mask(context, non_zero)
        targets = tf.fill([tf.shape(context)[0]], target)

        return tf.data.Dataset.from_tensor_slices((targets, context))

    return tf.data.Dataset.from_tensor_slices(positions).flat_map(extract_context)

# Apply skipgram generation to each vectorized sentence
sg_ds = vectorized_ds.flat_map(lambda x: generate_skipgram_pairs(x, window_size=4))

# Optionally convert to list of tuples
print(sg_ds.element_spec)
next(sg_ds.as_numpy_iterator())

(TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))


(np.int64(284), np.int64(17))

In [38]:
for target_idx, context_idx in sg_ds:
  print(f"target: {vectorize_layer.get_vocabulary()[target_idx]} | context: {vectorize_layer.get_vocabulary()[context_idx]}")

target: id | context: have
target: id | context: responded
target: id | context: if
target: id | context: i
target: have | context: id
target: have | context: responded
target: have | context: if
target: have | context: i
target: have | context: were
target: responded | context: id
target: responded | context: have
target: responded | context: if
target: responded | context: i
target: responded | context: were
target: responded | context: going
target: if | context: id
target: if | context: have
target: if | context: responded
target: if | context: i
target: if | context: were
target: if | context: going
target: i | context: id
target: i | context: have
target: i | context: responded
target: i | context: if
target: i | context: were
target: i | context: going
target: were | context: have
target: were | context: responded
target: were | context: if
target: were | context: i
target: were | context: going
target: going | context: responded
target: going | context: if
target: going | conte

In [8]:
train_df["text"].values[0], vectorize_layer(train_df["text"].values[0])

(' I`d have responded, if I were going',
 <tf.Tensor: shape=(7,), dtype=int64, numpy=array([5, 7, 3, 4, 6, 2, 8])>)

In [10]:
sg = []
sentence = train_df["text"].values[0]
sg_pairs, _ = tf.keras.preprocessing.sequence.skipgrams(
    vectorize_layer(sentence),
    vocab_size,
    window_size=4,
    negative_samples=1.0,
    shuffle=False,
    categorical=False,
    sampling_table=None,
    seed=None
)
sg.extend(sg_pairs)
sg = sg_pairs
type(sg), len(sg)

(list, 72)

In [11]:
for target_idx, context_idx in sg:
  try:
    print(f"target: {vectorize_layer.get_vocabulary()[target_idx]} | context: {vectorize_layer.get_vocabulary()[context_idx]}")
  except:
    print(target_idx, context_idx)
    pass

target: id | context: have
target: id | context: responded
target: id | context: if
target: id | context: i
target: have | context: id
target: have | context: responded
target: have | context: if
target: have | context: i
target: have | context: were
target: responded | context: id
target: responded | context: have
target: responded | context: if
target: responded | context: i
target: responded | context: were
target: responded | context: going
target: if | context: id
target: if | context: have
target: if | context: responded
target: if | context: i
target: if | context: were
target: if | context: going
target: i | context: id
target: i | context: have
target: i | context: responded
target: i | context: if
target: i | context: were
target: i | context: going
target: were | context: have
target: were | context: responded
target: were | context: if
target: were | context: i
target: were | context: going
target: going | context: responded
target: going | context: if
target: going | conte

In [49]:
import tensorflow as tf

def generate_skipgram_pairs(sequence, window_size):
    seq_len = tf.shape(sequence)[0]
    positions = tf.range(seq_len)

    def extract_context(i):
        target = sequence[i]

        # Skip if target is 0
        return tf.cond(
            tf.equal(target, 0),
            lambda: tf.data.Dataset.from_tensors((tf.constant(-1, dtype=sequence.dtype), tf.constant(-1, dtype=sequence.dtype))).take(0),
            lambda: _valid_context(i, target)
        )

    def _valid_context(i, target):
        start = tf.maximum(0, i - window_size)
        end = tf.minimum(seq_len, i + window_size + 1)

        # Exclude the center word itself
        left = sequence[start:i]
        right = sequence[i + 1:end]
        context = tf.concat([left, right], axis=0)

        # Remove context tokens that are 0
        non_zero_mask = tf.not_equal(context, 0)
        context = tf.boolean_mask(context, non_zero_mask)

        targets = tf.fill([tf.shape(context)[0]], target)

        return tf.data.Dataset.from_tensor_slices((targets, context))

    return tf.data.Dataset.from_tensor_slices(positions).flat_map(extract_context)


In [40]:
import tensorflow as tf

def generate_skipgram_pairs(sequence, window_size):
    seq_len = tf.shape(sequence)[0]
    positions = tf.range(seq_len)

    def extract_context(i):
        start = tf.maximum(0, i - window_size)
        end = tf.minimum(seq_len, i + window_size + 1)

        # Exclude the center word itself
        left = sequence[start:i]
        right = sequence[i + 1:end]
        context = tf.concat([left, right], axis=0)

        target = sequence[i]
        targets = tf.fill([tf.shape(context)[0]], target)

        return tf.data.Dataset.from_tensor_slices((targets, context))

    return tf.data.Dataset.from_tensor_slices(positions).flat_map(extract_context)


In [53]:
train_ds = (
    train_ds
    .map(
        # lambda x: ((tf.one_hot(x[0], depth=vocab_size), tf.one_hot(x[1], depth=vocab_size)), tf.constant(1.0)),
        lambda x: (x, tf.constant(1)),
        num_parallel_calls=tf.data.AUTOTUNE,
    )
)

print(train_ds.element_spec)
next(train_ds.as_numpy_iterator())

(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))


(b' I`d have responded, if I were going', np.int32(1))

In [54]:
tf.data.experimental.cardinality(train_ds).numpy()

np.int64(27480)

In [ ]:
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dot, Activation

embedding_dims = [45, 312, 752]
window_sizes   = [2, 4]
histories      = {}
for w in window_sizes:
    for ed in embedding_dims:
        # One-hot inputs (shape = (vocab_size,))
        input_target = Input(shape=(vocab_size,))
        input_context = Input(shape=(vocab_size,))

        # Shared embedding layer simulated with Dense layer
        embedding_layer = Dense(ed, use_bias=False)

        target_embedding = embedding_layer(input_target)   # shape: (embedding_dim,)
        context_embedding = embedding_layer(input_context) # same

        # Dot product of embeddings
        dot_product = Dot(axes=-1)([target_embedding, context_embedding])

        # Sigmoid output for skipgram-style binary prediction
        output = Activation('sigmoid')(dot_product)

        model = Model(inputs=[input_target, input_context], outputs=output)
        model.compile(optimizer='adam', loss='binary_crossentropy')

        history = model.fit(train_ds.batch(128).prefetch(tf.data.AUTOTUNE), epochs=5)

        histories[(w, ed)] = history

list(histories.keys())

Epoch 1/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - loss: 0.6577
Epoch 2/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - loss: 0.3212
Epoch 3/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - loss: 0.1664
Epoch 4/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 0.1053
Epoch 5/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.0723
Epoch 1/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 30s 124ms/step - loss: 0.5870
Epoch 2/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 29s 121ms/step - loss: 0.1609
Epoch 3/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 29s 121ms/step - loss: 0.0669
Epoch 4/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 29s 121ms/step - loss: 0.0338
Epoch 5/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 29s 121ms/step - loss: 0.0191
Epoch 1/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 68s 280ms/step - loss: 0.5400
Epoch 2/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 68s 280ms/step - loss: 0.1121
Epoch 3/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 66s 274ms/step - loss: 0.0383
Epoch 4/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 68s 279ms/step - loss: 0.0167
Epoch 5/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 67s 2

[(2, 45), (2, 312), (2, 752), (4, 45), (4, 312), (4, 752)]

In [ ]:
embeddings = embedding_layer.get_weights()[0]

print(embeddings.shape)

(8157, 128)


In [ ]:
# prompt: given vectorize_layer how to obtain the index for the word "what"

import numpy as np

# Get the vocabulary from the vectorize_layer
vocabulary = vectorize_layer.get_vocabulary()

# Find the index of the word "what"
try:
    what_index = vocabulary.index("what")
    print(f"The index of 'what' in the vocabulary is: {what_index}")
except ValueError:
    print("'what' is not found in the vocabulary.")


The index of 'what' in the vocabulary is: 52


In [ ]:
embeddings[what_index].shape

(128,)

In [ ]:
print(train_ds.element_spec)
next(train_ds.as_numpy_iterator())

((TensorSpec(shape=(8157,), dtype=tf.float32, name=None), TensorSpec(shape=(8157,), dtype=tf.float32, name=None)), TensorSpec(shape=(), dtype=tf.float32, name=None))


((array([0., 0., 1., ..., 0., 0., 0.], dtype=float32),
  array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)),
 np.float32(1.0))

In [ ]:
classifier_train_ds = tf.data.Dataset.from_tensor_slices((vectorized_train, df_train["sentiment"].values))

print(classifier_train_ds.element_spec)
next(classifier_train_ds.as_numpy_iterator())

(TensorSpec(shape=(33,), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))


(array([  293,    17, 15185,    69,     2,   120,    47,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0]),
 b'neutral')

In [ ]:
embedding_layer = tf.keras.layers.Embedding(
    input_dim=embeddings.shape[0],
    output_dim=embeddings.shape[1],
    weights=[embeddings],
    trainable=False
)

def embed_and_flatten(indices, label):
    embedded = embedding_layer(indices)        # shape: (seq_len, embed_dim)
    flat = tf.reshape(embedded, [-1])          # shape: (seq_len * embed_dim,)
    return flat, label

In [ ]:
classifier_train_ds = classifier_train_ds.map(embed_and_flatten)
print(classifier_train_ds.element_spec)
next(classifier_train_ds.as_numpy_iterator())

(TensorSpec(shape=(4224,), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))


(array([-0.17132318, -0.15126888, -0.17530611, ...,  0.4272256 ,
        -0.41871053, -0.40135565], dtype=float32),
 b'neutral')

In [ ]:
# Define a lookup table to convert string → integer
label_lookup = tf.keras.layers.StringLookup(
    vocabulary=df_train["sentiment"].unique().tolist(),
    num_oov_indices=0,
)

# Optional: one-hot encode
num_classes = label_lookup.vocabulary_size()

classifier_train_ds = classifier_train_ds.map(lambda x, y: (x, tf.one_hot(label_lookup(y), depth=num_classes)))

print(classifier_train_ds.element_spec)
next(classifier_train_ds.as_numpy_iterator())

(TensorSpec(shape=(4224,), dtype=tf.float32, name=None), TensorSpec(shape=(3,), dtype=tf.float32, name=None))


(array([-0.17132318, -0.15126888, -0.17530611, ...,  0.4272256 ,
        -0.41871053, -0.40135565], dtype=float32),
 array([1., 0., 0.], dtype=float32))

In [ ]:
classifier_train_ds = classifier_train_ds.batch(128)

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3-class classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(classifier_train_ds, epochs=3)


Epoch 1/3
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.3925 - loss: 0.6410
Epoch 2/3
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.4302 - loss: 0.6215
Epoch 3/3
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.4364 - loss: 0.6138
